# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [1]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.fasttext import FastText

In [2]:
path_to_dir = '/media/zu_ann/OS/Users/zu_ann/Yandex.Disk/HSE/DH/dh/araneum_none_fasttextskipgram_300_5_2018/'
model_path = 'araneum_none_fasttextskipgram_300_5_2018.model'

In [3]:
w2v_model = FastText.load(path_to_dir + model_path)

In [4]:
w2v_model.init_sims(replace=True)

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [5]:
import string
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from judicial_splitter import splitter
from tqdm import tqdm_notebook
from search_by_inverted_index import *

In [6]:
mystem = Mystem()

In [7]:
def preprocessing(mystem, input_text, del_stopwords=True, del_digit=False):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation + '»«–…—') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]
    
    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [8]:
def get_w2v_vectors(model, lemmas):
    """Получает вектор документа"""
    vec_list = []
    
    for word in lemmas:
        try:
            vec = model.wv[word]
            vec_list.append(vec)
        except:
            continue
            
    final_vec = sum(vec_list) / len(vec_list)
    return final_vec

def save_w2v_base(files_list, model, mystem, save=True):
    """Индексирует всю базу для поиска через word2vec"""
    documents_info = []    
    
    for file in tqdm_notebook(files_list):
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            lemmas = preprocessing(mystem, text)
            vec = get_w2v_vectors(model, lemmas)
            
            file_info = {'file': file, 'word2vec': vec}
            documents_info.append(file_info)
    
    if save:
        with open('w2v_base.pkl', 'wb') as fw:
            pickle.dump(documents_info, fw)
    
    return documents_info

In [9]:
path_to_files = '/media/zu_ann/OS/Users/zu_ann/Yandex.Disk/HSE/IR/articles/'
files_list = list(map(lambda x: path_to_files + x, os.listdir(path_to_files)[:1000]))

In [10]:
%%time

data_word2vec = save_w2v_base(files_list, w2v_model, mystem)

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))


CPU times: user 45 s, sys: 8.29 s, total: 53.3 s
Wall time: 1min 41s


## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [11]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [13]:
def get_paragraphs(files_list, mystem):
    file_text = {}
    data = []
    
    for file in files_list:
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            file_text[file] = text
            paragraphs = splitter(text, 1)
            
            for paragraph in paragraphs:
                paragraph_lemmatized = preprocessing(mystem, paragraph, del_stopwords=False)
                data.append({'file': file, 'paragraph': paragraph_lemmatized})
                
    with open('file_text', 'w', encoding='utf-8') as fw:
        json.dump(file_text, fw)
    
    return data, file_text

In [14]:
def train_doc2vec(data, epochs):
    tagged_data = [TaggedDocument(words=elem['paragraph'],
                                  tags=[str(i)]) for i, elem in enumerate(data)]
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, 
                min_alpha=0.025, epochs=epochs, workers=4, dm=1)
    
    model.build_vocab(tagged_data)
    print(len(model.wv.vocab))
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    return model

In [15]:
paragraphs, file_text = get_paragraphs(files_list, mystem)

In [16]:
%%time

d2v_model = train_doc2vec(paragraphs, 500)

3817
CPU times: user 15min 42s, sys: 2min 39s, total: 18min 21s
Wall time: 11min 50s


### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [17]:
def get_d2v_vectors(model, lemmas):
    """Получает вектор документа"""
    vec = model.infer_vector(lemmas)
    return vec
    
def save_d2v_base(model, paragraphs, save=True):
    """Индексирует всю базу для поиска через doc2vec"""
    documents_info = []    
    
    for paragraph in paragraphs:
        vec = get_d2v_vectors(model, paragraph['paragraph'])
            
        file_info = {'file': paragraph['file'], 'doc2vec': vec}
        documents_info.append(file_info)
    
    if save:
        with open('d2v_base.pickle', 'wb') as fw:
            pickle.dump(documents_info, fw)
    
    return documents_info 

In [18]:
data_doc2vec = save_d2v_base(d2v_model, paragraphs)

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [19]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [20]:
def search_w2v(query, w2v_model, data_word2vec, n_results):
    vec1 = get_w2v_vectors(w2v_model, query)
    similarity_dict = {}
    
    for elem in data_word2vec:
        sim = similarity(vec1, elem['word2vec'])
        similarity_dict[sim] = elem['file']
        
    relevant = [similarity_dict[sim] for sim in sorted(similarity_dict, reverse=True)[:n_results]]
    return relevant

def search_d2v(query, d2v_model, data_doc2vec, n_results):
    vec1 = get_d2v_vectors(d2v_model, query)
    similarity_dict = {}
    
    for elem in data_doc2vec:
        sim = similarity(vec1, elem['doc2vec'])
        similarity_dict[sim] = elem['file']
        
    relevant = [similarity_dict[sim] for sim in sorted(similarity_dict, reverse=True)[:n_results]]
    return relevant

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [21]:
%%time

inverted_index, document_length = get_inverted_index(mystem, files_list)

CPU times: user 44 s, sys: 7.32 s, total: 51.3 s
Wall time: 1min 39s


In [22]:
with open('inverted_index.json', 'r', encoding='utf-8') as f:
    inverted_index = json.load(f)

In [23]:
with open('document_length.json', 'r', encoding='utf-8') as f:
    document_length = json.load(f)

In [22]:
def search(query, search_method, n_results=5):
    
    if search_method == 'inverted_index':
        search_result = get_search_result(query, inverted_index, mystem, files_list, document_length, n_results)
    
    elif search_method == 'word2vec':
        preprocessed_query = preprocessing(mystem, query)
        search_result = search_w2v(preprocessed_query, w2v_model, data_word2vec, n_results)
    
    elif search_method == 'doc2vec':
        preprocessed_query = preprocessing(mystem, query, del_stopwords=False)
        search_result = search_d2v(preprocessed_query, d2v_model, data_doc2vec, n_results)
    
    else:
        raise TypeError('unsupported search method')
    
    results = [(filename, file_text[filename]) for filename in search_result]
    return results

In [23]:
search('отчет об итогах выпуска обыкновенных именных бездокументарных акций', 'inverted_index')

[('/media/zu_ann/OS/Users/zu_ann/Yandex.Disk/HSE/IR/articles/100429.txt',
  '\n\nКоллегия судей Высшего Арбитражного Суда Российской Федерации в составе председательствующего судьи  Новоселовой  Л.А., судей Иванниковой Н.П. и Киреева Ю.А. рассмотрела в судебном заседании заявление  Дербинцевой  С.С. (г.  Партизанск  Приморского края) о пересмотре в порядке надзора решения Арбитражного суда Приморского края от 03.12.07 по делу N А51-9998/07-29-392, постановления суда апелляционной инстанции от 28.10.08 и постановления Федерального   арбитражного суда Дальневосточного округа от 23.01.09 по тому же делу по заявлению  Дербинцевой  С.С. о признании незаконными приказов регионального отделения Федеральной службы по финансовым рынкам в Дальневосточном федеральном округе (г. Владивосток) от 19.01.05 N 34-п о государственной регистрации выпуска обыкновенных именных бездокументарных акций ОАО "Универмаг "Центральный" (г.  Партизанск ) в количестве 10 705 штук, размещенных путем конвертации обыкн

In [24]:
search('отчет об итогах выпуска обыкновенных именных бездокументарных акций', 'word2vec')

[('/media/zu_ann/OS/Users/zu_ann/Yandex.Disk/HSE/IR/articles/100155.txt',
  '\n\nКоллегия судей Высшего Арбитражного Суда Российской Федерации в составе: председательствующего судьи Киреева Ю.А., судей Иванниковой Н.П. и  Новоселовой  Л.А. рассмотрела в судебном заседании заявление открытого акционерного общества "Завод "ЭКОМАШ" (г. Люберцы), граждан Брегмана В.К. (г. Люберцы) и  Чакветадзе  Ф.А. (г. Люберцы) от 28.04.2009 N 20и о пересмотре в порядке надзора решения Арбитражного суда Московской области   от 08.08.2008 по делу N А41-К1-11758/07 и постановления Федерального арбитражного суда Московского округа от 09.02.2009 по тому же делу по иску граждан Копыленко Б.А. (г. Москва), Макеева Ю.М. (г. Люберцы), Шейниса Л.П. (г. Люберцы) к открытому акционерному обществу "Завод "ЭКОМАШ" (далее - общество "Завод "ЭКОМАШ", общество) (г. Люберцы), закрытому акционерному обществу "РДЦ "Паритет" (г. Москва), гражданам Брегману  В.К. (г. Люберцы) и  Чакветадзе  Ф.А. (г. Люберцы) о признании за и

In [25]:
search('отчет об итогах выпуска обыкновенных именных бездокументарных акций', 'doc2vec')

[('/media/zu_ann/OS/Users/zu_ann/Yandex.Disk/HSE/IR/articles/100153.txt',
  '\n\nКоллегия судей Высшего Арбитражного Суда Российской Федерации в составе председательствующего судьи Чистякова А.И., судей Андреева Е.И., Александрова В.Н., рассмотрев в судебном заседании заявление Территориального управления Федеральной службы финансово-бюджетного надзора в Волгоградской области о пересмотре в порядке надзора постановления Двенадцатого арбитражного апелляционного суда от 17.11.2008 постановления Федерального арбитражного суда Поволжского округа от 05.02.2009 по делу N А12-12072/08-С45  Арбитражного суда Волгоградской области, установила: общество с ограниченной ответственностью " Промобслуживание -С" (далее - общество, г. Волгоград) обратилось в Арбитражный суд Волгоградской области с заявлением о признании незаконным и отмене постановления Территориального управления Федеральной службы финансово-бюджетного надзора в Волгоградской области (далее - управление, г. Волгоград) от 26.06.2008 N